In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [2]:
count=0
data=pd.read_csv("../merged/len4_ndb.csv",encoding="shift-jis",index_col=0)
# drop never treated group
data=data.dropna(subset=["Approved"])
data.head()

C:\Users\hakos\AppData\Local\Temp\ipykernel_19140\2597341367.py:2: DtypeWarning: Columns (14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.read_csv("../merged/len4_ndb.csv",encoding="shift-jis",index_col=0)


,薬効分類,薬効分類名称,医薬品コード,医薬品名,薬価基準収載医薬品コード,薬価,後発品区分,総計,year,in_hospital,oral,id_l4,Approved,KEGG,Ingredient,variable,id,revenue
0,114.0,解熱鎮痛消炎剤,661140242.0,インテバン坐剤５０ ５０ｍｇ,1145700J3012,19.3,0.0,292190.95,2014,1,0,1145,1985.0,D00141,Indometacin,id1,1145.0,5.639285e+06
1,114.0,解熱鎮痛消炎剤,661140216.0,インテバン坐剤２５ ２５ｍｇ,1145700J2016,19.3,0.0,128832.00,2014,1,0,1145,1985.0,D00141,Indometacin,id1,1145.0,2.486458e+06
2,114.0,解熱鎮痛消炎剤,661140020.0,インドメタシン坐剤 ５０ｍｇ,1145700J3012,19.3,0.0,106328.00,2014,1,0,1145,1985.0,D00141,Indometacin,id1,1145.0,2.052130e+06
3,114.0,解熱鎮痛消炎剤,661140242.0,インテバン坐剤５０ ５０ｍｇ,1145700J3012,19.3,0.0,658257.00,2014,0,0,1145,1985.0,D00141,Indometacin,id1,1145.0,1.270436e+07
4,114.0,解熱鎮痛消炎剤,661140216.0,インテバン坐剤２５ ２５ｍｇ,1145700J2016,19.3,0.0,418727.50,2014,0,0,1145,1985.0,D00141,Indometacin,id1,1145.0,8.081441e+06


1 observation が複数行にまたがっているので経過年数ダミー以外は落として1行にまとめよう

In [3]:
data["elasped"]=data["year"]-data["Approved"]
#　不要なところを除外
# data.loc[(data['elasped'] > 10) | (data["elasped"]< -5), 'elasped'] = np.nan

# dummy をとりたいところを分離
dummies=pd.get_dummies(data["elasped"],prefix="elasped")
dummies.columns = dummies.columns.str.replace('-', 'm').str.replace(r'\..*', '', regex=True)
# sumをとりたいところを分離
sumcolumns=["総計","revenue"]
sums=data.groupby(["year","id","後発品区分","in_hospital","oral"])[sumcolumns].sum().reset_index()
# save temporarily
colnames=dummies.columns.to_list()
if count:
# count ver.
    aggregated_df = pd.concat([
        data.groupby(['year', 'id',"後発品区分","in_hospital","oral"]).first(), 
        dummies.groupby([data['year'], data['id'],data["後発品区分"],data["in_hospital"],data["oral"]]).sum()
    ], axis=1).reset_index()
else: 
    # dummy ver.
    aggregated_df = pd.concat([
        data.groupby(['year', 'id',"後発品区分","in_hospital","oral"]).first(), 
        dummies.groupby([data['year'], data['id'],data["後発品区分"],data["in_hospital"],data["oral"]]).apply(lambda x: (x[colnames] > 0).any().astype(int))
    ], axis=1).reset_index()
# sumをとったところを追加する
aggregated_df[sumcolumns]=sums[sumcolumns]
aggregated_df
aggregated_df.head()

,year,id,後発品区分,in_hospital,oral,薬効分類,薬効分類名称,医薬品コード,医薬品名,薬価基準収載医薬品コード,...,elasped_29,elasped_30,elasped_31,elasped_32,elasped_33,elasped_34,elasped_35,elasped_36,elasped_37,elasped_38
0,2014,1145.0,0.0,0,0,114.0,解熱鎮痛消炎剤,661140242.0,インテバン坐剤５０ ５０ｍｇ,1145700J3012,...,1,0,0,0,0,0,0,0,0,0
1,2014,1145.0,0.0,1,0,114.0,解熱鎮痛消炎剤,661140242.0,インテバン坐剤５０ ５０ｍｇ,1145700J3012,...,1,0,0,0,0,0,0,0,0,0
2,2014,1147.0,0.0,0,0,114.0,解熱鎮痛消炎剤,661140080.0,ボルタレンサポ２５ｍｇ,1147700J2061,...,0,0,0,0,0,0,0,0,0,0
3,2014,1147.0,0.0,0,1,114.0,解熱鎮痛消炎剤,620007096.0,ボルタレン錠２５ｍｇ,1147002F1560,...,0,0,0,0,0,0,0,0,0,0
4,2014,1147.0,0.0,1,0,114.0,解熱鎮痛消炎剤,661140081.0,ボルタレンサポ５０ｍｇ,1147700J3084,...,0,0,0,0,0,0,0,0,0,0


connect generic usage

In [4]:
dusage=pd.read_csv("../generic/generic_usage_imp.csv")
dusage
# generic from ndb
dgeneric=pd.read_csv("../merged/len4_ndb_generic_imputed.csv",index_col=0,encoding="shift-jis")
dgeneric

,id,year,generic_share_q,generic_share_r
0,1145.0,2014,0.000000,0.000000
5,1147.0,2014,0.076961,0.059379
38,1149.0,2014,0.244198,0.053661
85,1219.0,2014,0.062114,0.053451
101,1231.0,2014,0.129137,0.064243
...,...,...,...,...
28997,4413.0,2021,0.342768,0.263456
29037,4490.0,2021,0.608051,0.363450
29241,6250.0,2021,0.502607,0.052158
29483,6290.0,2021,0.284349,0.073842


In [5]:
merged_df=aggregated_df.merge(dusage,left_on="year",right_on="Year")
merged_df=merged_df.merge(dgeneric,left_on=["id","year"],right_on=["id","year"])
merged_df.head()

,year,id,後発品区分,in_hospital,oral,薬効分類,薬効分類名称,医薬品コード,医薬品名,薬価基準収載医薬品コード,...,elasped_33,elasped_34,elasped_35,elasped_36,elasped_37,elasped_38,Year,Percentage,generic_share_q,generic_share_r
0,2014,1145.0,0.0,0,0,114.0,解熱鎮痛消炎剤,661140242.0,インテバン坐剤５０ ５０ｍｇ,1145700J3012,...,0,0,0,0,0,0,2014,51.55,0.000000,0.000000
1,2014,1145.0,0.0,1,0,114.0,解熱鎮痛消炎剤,661140242.0,インテバン坐剤５０ ５０ｍｇ,1145700J3012,...,0,0,0,0,0,0,2014,51.55,0.000000,0.000000
2,2014,1147.0,0.0,0,0,114.0,解熱鎮痛消炎剤,661140080.0,ボルタレンサポ２５ｍｇ,1147700J2061,...,0,0,0,0,0,0,2014,51.55,0.076961,0.059379
3,2014,1147.0,0.0,0,1,114.0,解熱鎮痛消炎剤,620007096.0,ボルタレン錠２５ｍｇ,1147002F1560,...,0,0,0,0,0,0,2014,51.55,0.076961,0.059379
4,2014,1147.0,0.0,1,0,114.0,解熱鎮痛消炎剤,661140081.0,ボルタレンサポ５０ｍｇ,1147700J3084,...,0,0,0,0,0,0,2014,51.55,0.076961,0.059379


In [6]:
id_dummies=pd.get_dummies(merged_df["id"],prefix="id")
df_with_dummies=pd.concat([merged_df,id_dummies],axis=1)
df_with_dummies.rename(columns={"Percentage":"generic_per"},inplace=True)
df_with_dummies.head()

,year,id,後発品区分,in_hospital,oral,薬効分類,薬効分類名称,医薬品コード,医薬品名,薬価基準収載医薬品コード,...,id_3327.0,id_3399.0,id_3919.0,id_3992.0,id_3999.0,id_4413.0,id_4490.0,id_6250.0,id_6290.0,id_7990.0
0,2014,1145.0,0.0,0,0,114.0,解熱鎮痛消炎剤,661140242.0,インテバン坐剤５０ ５０ｍｇ,1145700J3012,...,0,0,0,0,0,0,0,0,0,0
1,2014,1145.0,0.0,1,0,114.0,解熱鎮痛消炎剤,661140242.0,インテバン坐剤５０ ５０ｍｇ,1145700J3012,...,0,0,0,0,0,0,0,0,0,0
2,2014,1147.0,0.0,0,0,114.0,解熱鎮痛消炎剤,661140080.0,ボルタレンサポ２５ｍｇ,1147700J2061,...,0,0,0,0,0,0,0,0,0,0
3,2014,1147.0,0.0,0,1,114.0,解熱鎮痛消炎剤,620007096.0,ボルタレン錠２５ｍｇ,1147002F1560,...,0,0,0,0,0,0,0,0,0,0
4,2014,1147.0,0.0,1,0,114.0,解熱鎮痛消炎剤,661140081.0,ボルタレンサポ５０ｍｇ,1147700J3084,...,0,0,0,0,0,0,0,0,0,0


In [7]:
# df_with_dummies.to_csv("len3_agg.csv",encoding="shift-jis")
if not(count):
    # dummy ver.
    df_with_dummies.to_csv("../merged/len4_ndb_agg_in_oral_DropNever.csv",encoding="shift-jis")
else:
    # count ver.
    df_with_dummies.to_csv("../merged/len4_ndb_sum_agg_in_oral_DropNever.csv",encoding="shift-jis")